In [1]:
from bs4 import BeautifulSoup
import requests
import time
import pandas as pd
import re
import os

import selenium
from selenium import webdriver
from pyshadow.main import Shadow
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, ElementClickInterceptedException, NoSuchElementException
from selenium.webdriver.chrome.service import Service

headers = {'User-Agent': 'Mozilla/5.0'}

In [19]:
carmax_sf_url = "https://www.carmax.com/cars/all?location=san+francisco+ca"

driver = webdriver.Chrome(executable_path='/Users/xbn/Desktop/Data Design & Representation/HW/Final Project/chromedriver')
driver.get(carmax_sf_url)

WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "hzn-button")))

# Click the "See More Matches" button until the end is reached
while True:
    try:
        shadow_host = driver.find_element(By.CSS_SELECTOR, "hzn-button")
        shadow_root = driver.execute_script('return arguments[0].shadowRoot', shadow_host)
        button = shadow_root.find_element(By.CSS_SELECTOR, "button")

        # Scroll the button into view
        driver.execute_script("arguments[0].scrollIntoView(true);", button)

        # Attempt to click the button
        button.click()
    
    except ElementClickInterceptedException:
        # If normal click doesn't work, try clicking with JavaScript
        driver.execute_script("arguments[0].click();", button)

# Save the page source to a file
with open("carmax_sf.htm", "w", encoding="utf-8") as f:
    f.write(driver.page_source)

# Close the driver
driver.quit()

AttributeError: 'dict' object has no attribute 'find_element'

In [24]:
carmax_sf_url = "https://www.carmax.com/cars/all?location=san+francisco+ca"

driver = webdriver.Chrome(executable_path='/Users/xbn/Desktop/Data Design & Representation/HW/Final Project/chromedriver')
driver.get(carmax_sf_url)

WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "hzn-button")))

# Click the "See More Matches" button until the end is reached
while True:
    try:
        button = driver.find_element(By.CSS_SELECTOR, "button.hzn-button")
        button.click()

        # Wait a moment for the page to load more items
        time.sleep(5)

    except (TimeoutException) as e:
        # If the button is no longer present, or a timeout occurs, we break out of the loop
        print("Reached the end or encountered an error:",e)
        break

# Save the page source to a file
with open("carmax_sf.htm", "w", encoding="utf-8") as f:
    f.write(driver.page_source)

# Close the driver
driver.quit()

TimeoutException: Message: 


In [23]:
carmax_sf_url = "https://www.carmax.com/cars/all?location=san+francisco+ca"

driver = webdriver.Chrome(executable_path='/Users/xbn/Desktop/Data Design & Representation/HW/Final Project/chromedriver')
driver.get(carmax_sf_url)

try:
    # Wait for the shadow host to be present
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CLASS_NAME, "hzn-button"))
    )
    shadow_host = driver.find_element(By.CLASS_NAME, "hzn-button")
    shadow_root = driver.execute_script('return arguments[0].shadowRoot', shadow_host)

    # Click the "See More Matches" button until the end is reached
    while True:
        try:
            button = shadow_root.find_element(By.CSS_SELECTOR, "button")
            button.click()
            # Wait for new content to load here, if necessary
            WebDriverWait(driver, 10).until(
                lambda d: d.find_element(By.CLASS_NAME, "loading-indicator").get_attribute('style') == 'display: none;'
            )
        except NoSuchElementException:
            # If the button is no longer found, we've reached the end
            break
        except ElementClickInterceptedException:
            # If click is intercepted, you might need to wait or scroll
            pass

except TimeoutException:
    print("The hzn-button element was not found within the given time.")

# Save the page source to a file
with open("carmax_sf.htm", "w", encoding="utf-8") as f:
    f.write(driver.page_source)

# Close the driver
driver.quit()

The hzn-button element was not found within the given time.


In [6]:
carmax_sf_url = "https://www.carmax.com/cars/all?location=san+francisco+ca"

driver = webdriver.Chrome(executable_path='/Users/xbn/Desktop/Data Design & Representation/HW/Final Project/chromedriver')
driver.get(carmax_sf_url)
shadow = Shadow(driver)

wait = WebDriverWait(driver, 20)
def get_shadow_root(element):
    return driver.execute_script('return arguments[0].shadowRoot', element)

# Click the "See More Products" button until it's no longer found
while True:
    try:
        shadow_host = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="see-more"]//hzn-button')))
        see_more = get_shadow_root(shadow_host).find_element(By.CSS_SELECTOR, 'button.hzn-button')
        see_more.click()
        
    except TimeoutException:
        # If the button is no longer clickable, exit the loop
        print("No more 'See More Products' button to click.")
        break

with open("carmax_sf.htm", "w", encoding="utf-8") as f:
        f.write(driver.page_source)

driver.quit()

AttributeError: 'dict' object has no attribute 'find_element'

In [9]:
carmax_sf_url = "https://www.carmax.com/cars/all?location=san+francisco+ca"

driver = webdriver.Chrome(executable_path='/Users/xbn/Desktop/Data Design & Representation/HW/Final Project/chromedriver')
driver.get(carmax_sf_url)
shadow = Shadow(driver)

def click_shadow_button(driver, shadow_host_selector, button_selector):
    # Use JavaScript to click the button within the shadow root
    script = """
    var shadowHost = arguments[0];
    var shadowRoot = shadowHost.shadowRoot;
    var button = shadowRoot.querySelector(arguments[1]);
    if (button) {
        button.click();
        return true;
    }
    return false;
    """
    shadow_host = driver.find_element(By.CSS_SELECTOR, shadow_host_selector)
    return driver.execute_script(script, shadow_host, button_selector)

wait = WebDriverWait(driver, 20)

# Click the "See More Products" button until it's no longer found
while True:
    try:
        # Check if the shadow host is present
        shadow_host_selector = 'hzn-button'
        button_selector = 'button.hzn-button'
        
        # Click the button within the shadow root
        result = click_shadow_button(driver, shadow_host_selector, button_selector)
        
        if not result:
            print("The button was not found or not clickable.")
            break

        # Wait for some condition here that indicates the new content has been loaded
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'hzn-button')))
    except TimeoutException:
            # If we can't find the shadow host or the button, exit the loop
            print("No more 'See More Products' button to click.")
            break
with open("carmax_sf.htm", "w", encoding="utf-8") as f:
        f.write(driver.page_source)

driver.quit()

KeyboardInterrupt: 

In [31]:
carmax_sf_url = "https://www.carmax.com/cars/all?location=san+francisco+ca"

driver = webdriver.Chrome(executable_path='/Users/xbn/Desktop/Data Design & Representation/HW/Final Project/chromedriver')
driver.get(carmax_sf_url)

wait = WebDriverWait(driver, 20)
i=1

while True:
    try:
        # Save temporary page source
        with open("page/carmax_sf_{}.htm".format(i), "w", encoding="utf-8") as f:
            f.write(driver.page_source)

        # Scroll to the "See More Products" button to ensure it's in view
        see_more_button_presense = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "div#see-more hzn-button")))
        driver.execute_script("arguments[0].scrollIntoView(true);", see_more_button_presense)
        time.sleep(1)

        # Click the "See More Products" button
        see_more_button_clickable = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "div#see-more hzn-button")))
        driver.execute_script("arguments[0].click();", see_more_button_clickable)
        time.sleep(2)
        
        # After clicking, scroll to the bottom of the page to ensure all lazy-loaded content loads
        #driver.execute_script("window.scrollBy(0, 2500)")
        #driver.execute_script("arguments[0].scrollIntoView(true);", see_more_button_presense)
        
        # Wait for the page to load more products, adjust the wait condition as needed
        #wait.until(EC.presence_of_element_located((By.TAG_NAME, "hzn-button")))

        i+=1

    except (TimeoutException, ElementClickInterceptedException) as e:
        print("No more 'See More Products' button to click or Error encountered:\n", e)
        break

# Save whole page source
with open("carmax_sf_all.htm", "w", encoding="utf-8") as f:
    f.write(driver.page_source)

driver.quit()

No more 'See More Products' button to click or Error encountered:
 Message: script timeout
  (Session info: chrome=122.0.6261.94)



In [26]:
with open("carmax_sf.htm", "r", encoding="utf-8") as f:
    htm = f.read()

soup = BeautifulSoup(htm, 'html.parser')
print(soup.prettify())

<html class="js-focus-visible" data-js-focus-visible="" lang="en" style="--vh: 9.07px; --vw: 1200px;">
 <head class="at-element-marker">
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1.0" name="viewport"/>
  <title>
   Used cars in San Francisco, CA for Sale
  </title>
  <meta content="Used cars in San Francisco, CA for Sale on carmax.com. Search used cars, research vehicle models, and compare cars, all online at carmax.com" name="description"/>
  <meta name="keywords"/>
  <link href="https://www.carmax.com/cars/all?location=san+francisco+ca" rel="canonical"/>
  <meta content="index,follow" name="robots"/>
  <link href="//img2.carmax.com" rel="preconnect"/>
  <link href="//cdn.levelaccess.net" rel="preconnect"/>
  <link href="//googletagmanager.com" rel="preconnect"/>
  <link href="https://www.carmax.com" rel="preconnect"/>
  <link as="font" crossorigin="" href="https://www.carmax.com/shared/fonts/lato-v16-latin-regular.woff2" rel="preload" type="font/wof

In [28]:
links = soup.select('a[href]')
list = []
for link in links:
    list.append(link['href'])

print(len(list))

df_list = pd.DataFrame(list)
df_list.nunique()

197


0    104
dtype: int64

In [30]:
df_list.to_csv("product_links.csv", index=False)

In [ ]:
for link in link_list:
    time.sleep(5)
    page = requests.get(link, headers=headers)
    
    id = re.search(r'\d+(?=\.html)', link).group() # get id from url

    with open(f"HTML/{id}.html", 'w', encoding='utf-8') as file: # save in folder 'HTML'
        file.write(str(page.content)) # write HTML content in file

The body type of cars is not shown on its product page.

I need to use filter and 'See More' within each type

Nvm I'll just change my topic only focusing on SUVs...